In [257]:
import pandas as pd
# imports googlesearch to use requests and BS4 to scrape google
from googlesearch import search
import csv

In [258]:
# In the future, we can improve this by being able to read in any/all the datasets

# reads in the data set
#data = pd.read_csv(r"Affiliation_Parsing/NeurIPS affiliations/NeurIPSaffiliations-2010-affiliation.csv")
openfile =(r"/Users/jhanviganesh/Gender Research/ICML2019xad.csv")
data = pd.read_csv(openfile)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                20 non-null     int64 
 1   first-author              20 non-null     object
 2   first-author-affiliation  20 non-null     object
 3   last-author               20 non-null     object
 4   last-author-affiliation   20 non-null     object
 5   title                     20 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.1+ KB


In [259]:
# drops every column that is not the authors or their affiliations
#print(data.columns[1])
#data = data.drop(data.columns[5], axis=1)
#data = data.drop(data.columns[4], axis=1)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                20 non-null     int64 
 1   first-author              20 non-null     object
 2   first-author-affiliation  20 non-null     object
 3   last-author               20 non-null     object
 4   last-author-affiliation   20 non-null     object
 5   title                     20 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.1+ KB


In [260]:
# imports BS4 libraries for webscraping
from re import I
import requests
from bs4 import BeautifulSoup


In [244]:
# Need to add in they/them
# Improve search


def findGender(urlLink):
    gender = ''
    ## put a url where the desciption of the author is avilable
    
    url = urlLink
   
    htmlText = requests.get(url).text
    #print(htmlText)
    soup = BeautifulSoup(htmlText, 'html.parser')
    
    
    # gathers all the p tags from the html
    soupstring=str(soup.find_all("div")) + str(soup.find_all("p"))
    #print(soupstring)
    shetags = [" she.", " she, "," she ", " She ","She ", 
              " her.", " her, "," her "," Her ","Her "
              " her."," hers, "," hers ","Hers "," Hers"
               , "She’s ", " she's " ]
    
    hetags = [" he.", " he, "," he ", " He ","He ", 
              " his.", " his, "," his "," His ","His "
              " him."," him, "," him "," Him ", "He’s ", " he's " ]
    
    theytags = [" they ", "They ", " them ", "Them " ]

    ##checks if male or female pronouns are present in the desciption

    containsfemale = any(femalepronouns in soupstring for femalepronouns in shetags)
    containsmale = any(malepronouns in soupstring for malepronouns in hetags)
    containsthey = any(theypronouns in soupstring for theypronouns in theytags)

    if containsfemale:
        gender = "Female"
    elif containsmale:
        
        gender = "Male"
#     elif containsthey:
#         gender = "they/them"
    else:
        gender = "inconclusive"
    return  gender 



In [217]:
# returns whether or not a substring exists in a string
def substring_exists(string, sub_string):
    return string.find(sub_string)


In [218]:
# returns a list of the first-authors
def getFirstAuthorList(authorColum):
    for column in data[[authorColum]]:
        authors = data[column]

        '''
        Prints out column name and its values
        '''
        #print('Column Name: '+column)
        # print(authors.values)

    lst_authors = authors.values
    return lst_authors


In [230]:
def generateLinks(thecolum, theaffiliation, titleofnewCol):

    counter = 0
    linkArray = []
    lst = getFirstAuthorList(thecolum)
    print(lst)
    getAllLinks=[]
    
    #if we have any particular domains which we dont want to include in out searches we can input them in this list
    unwantedLinks = ["pdf", "dblp","dl.acm.org"]
    for person in lst:

        query = person+" " + data[theaffiliation].values[counter]+" .edu"
        #print(query)
        for link in search(query, num=10, stop=10):
            # appends .edu and .org links into the array
            if(substring_exists(link, ".edu") != -1 or substring_exists(link, ".org") != -1 ):
                #checks if each link contains anything from the list of unwanted list
                containsUnwanted = any(elements in link for elements in unwantedLinks)
                if(containsUnwanted == False):
                    linkArray.append(link)

  
        getAllLinks.append(linkArray)
        linkArray=[]
        counter += 1


        with open('newCSV.csv', 'w') as file:
            write = csv.writer(file)
            write.writerow([titleofnewCol])
            for val in getAllLinks:
                write.writerow([val])
        # with open('newCSV.csv','r') as file1, open(openfile,'r') as file2,open('combined.csv', 'w') as result:
        #     file1reader = csv.reader(file1)
        #     file2reader = csv.reader(file2)
        #     result = csv.writer(result)
        #     result.writerows(x + y for x, y in zip(file2reader,file1reader))

        

In [262]:
generateLinks("last-author", "last-author-affiliation","Last Author Links")

['Yossi Matias' 'Stephan Günnemann' 'David Lopez-Paz' 'Volkan Cevher'
 'Jun Zhu' 'Zico Kolter' 'Ilya Razenshteyn' 'Dahua Lin' 'Mantas Mazeika'
 'Elvis Dohmatob' 'Luca Daniel' 'Geoff Gordon' 'Daniel Soudry'
 'Piotr Majdak' 'Yaron Lipman' 'Ruosong Wang' 'Max Welling'
 'Timothy Hospedales' 'Daniel E Worrall' 'Raman Arora']


HTTPError: HTTP Error 429: Too Many Requests

In [252]:

def AuthorGender():

    fileNew = open("/Users/jhanviganesh/Gender Research/ICML2019xac.csv", "r")

    
    update = []
    reader = csv.DictReader(fileNew)

    for rowVal in reader:
        for x in [rowVal["First Author Links"]]:
            authorList = x.split("'")
            authorList = list(filter(('"[').__ne__, authorList))
            authorList = list(filter(('[').__ne__, authorList))
            authorList = list(filter((", ").__ne__, authorList))
            authorList = list(filter(("]").__ne__, authorList))
            authorList = list(filter((']"').__ne__, authorList))
            print(authorList)
            for link in authorList:
                if (link == '[]'):
                    FirstfinalGender = 'inconclusive'
                    break
                print('First THE LINK: '+ link)
                FirstfinalGender = findGender(link)
                if(FirstfinalGender == 'Male' or FirstfinalGender == 'Female'):
                    break   
                
        for x in [rowVal["Last Author Links"]]:
            authorList = x.split("'")
            authorList = list(filter(('"[').__ne__, authorList))
            authorList = list(filter(('[').__ne__, authorList))
            authorList = list(filter(('"').__ne__, authorList))
            authorList = list(filter((", ").__ne__, authorList))
            authorList = list(filter(("]").__ne__, authorList))
            authorList = list(filter((']"').__ne__, authorList))
            authorList = list(filter(('"[]"').__ne__, authorList))
            
            print(authorList)
            for link in authorList:
                if (link == '[]'):
                    LastfinalGender = 'inconclusive'
                    break
                print('Last THE LINK: '+ link)
                LastfinalGender = findGender(link)
                if(LastfinalGender == 'Male' or LastfinalGender == 'Female'):
                    break 
        row = {"first-author": rowVal["first-author"],"first-author-affiliation":rowVal["first-author-affiliation"],"First Author Gender": FirstfinalGender,"last-author":rowVal["last-author"],"last-author-affiliation":rowVal["last-author-affiliation"],"Last Author Gender": LastfinalGender,"title": rowVal["title"]}
        update.append(row)
    fileNew.close()
    op = open("FinalFinal.csv", "w", newline='')
    headers = ['first-author', 'first-author-affiliation', 'First Author Gender','last-author','last-author-affiliation','Last Author Gender','title']
    data = csv.DictWriter(op, delimiter=',', fieldnames=headers)
    data.writerow(dict((heads, heads) for heads in headers))
    data.writerows(update)
    op.close()
    

    

In [256]:
AuthorGender()

['[]']
['https://ece.duke.edu/faculty/lawrence-carin', 'https://today.duke.edu/2021/01/research-vp-carin-depart-provost-role-kaust', 'http://people.ee.duke.edu/~lcarin/', 'https://healthdata.pratt.duke.edu/people/lawrence-carin', 'https://scholars.duke.edu/person/lcarin', 'https://today.duke.edu/2019/08/carin-named-new-vice-president-research-university-wide-responsibilities', 'https://precisionmedicine.duke.edu/about/faculty/lawrence-carin', 'https://ai.meng.duke.edu/faculty/lawrence-carin', 'https://autismcenter.duke.edu/content/lawrence-carin-phd', 'https://www.kaust.edu.sa/en/news/lawrence-carin-named-kaust-provost']
Last THE LINK: https://ece.duke.edu/faculty/lawrence-carin
['https://www.chessprogramming.org/Arthur_Guez', 'https://www.aaai.org/ocs/index.php/AAAI/AAAI16/rt/metadata/12389/0']
First THE LINK: https://www.chessprogramming.org/Arthur_Guez
['https://en.wikipedia.org/wiki/Timothy_Lillicrap', 'https://en.wikipedia.org/wiki/Timothy_Lillicrap#Biography', 'https://en.wikiped